## Habitable Planet Data Cleaning

In [3]:
import pandas as pd
import numpy as np

print(G)
print(M_earth)
print(R_earth)


# m3 / (kg s2)
G = 6.67e-11

# kg
M_earth = 1.34e20

#
R_earth = 6378100


  Name   = Gravitational constant
  Value  = 6.6743e-11
  Uncertainty  = 1.5e-15
  Unit  = m3 / (kg s2)
  Reference = CODATA 2018
  Name   = Earth mass
  Value  = 5.972167867791379e+24
  Uncertainty  = 1.3422009501651213e+20
  Unit  = kg
  Reference = IAU 2015 Resolution B 3 + CODATA 2018
  Name   = Nominal Earth equatorial radius
  Value  = 6378100.0
  Uncertainty  = 0.0
  Unit  = m
  Reference = IAU 2015 Resolution B 3


In [ ]:
k_data = pd.read_csv("planet.csv")
k_data.columns = ['planet_name', 'num_stars', 'controv_flag', 'orbital_period', 'planet_semi-major_axis', 'planet_radius', 'planet_mass', 'eccentricity', 'insolation_flux', 'equi_temp', 'spectral_type', 'stellar_temp', 'stellar_radius', 'stellar_mass', 'distance']

planet_name = Name of Planet

num_stars = Number of stars

controv_flag = 0 if no controversy, else 1

orbital_period = orbital period [days]

planet_semi-major_axis = distance from star [au]

planet_radius = planet radius [Earth Radius]

planet_mass = planet mass [Earth Mass]

eccentricity = amount by which orbit of planet deviates from a perfect circle (instability in temp)

insolation_flux = Another form of equi_temp [Earth]

equi_temp = planetary equilibrium (The equilibrium temperature of the planet as modeled by a black body heated only by its host star, or for directly imaged planets, the effective temperature of the planet required to match the measured luminosity if the planet were a black body)

spectral_type = Classification based upon Morgan-Keenan system. (Hottest) O, B, A, F, G, K, and M (Coolest). Each letter class is then subdivided using a numeric digit with 0 being hottest and 9 being coolest. Luminosity class 0 or Ia+ is used for hypergiants, class I for supergiants, class II for bright giants, class III for regular giants, class IV for subgiants, class V for main-sequence stars, class sd (or VI) for subdwarfs, and class D (or VII) for white dwarfs.

stellar_radius = radius of star [Sol Radius]

stellar_mass = mass of star [Sol Mass]

distance = distance from Earth [parsec]

In [ ]:
k_data.head()

In [ ]:
#Total rows
k_data['planet_name'].count()

In [ ]:
#Total planets
k_data['planet_name'].nunique()

In [ ]:
#Take duplicates and average the data
avg_data = k_data.groupby('planet_name').mean(numeric_only=True).reset_index()
avg_data.reset_index(drop=True, inplace=True)
avg_data.tail()

In [ ]:
def first_non_zero(s):
    non_zero_values = s[s != '0']
    if len(non_zero_values) > 0:
        return non_zero_values.iloc[0]
    else:
        return '0'

In [ ]:
# Save Spectral_Type data
k_data['spectral_type'] = k_data['spectral_type'].fillna('0').astype(str)
spectral = k_data.groupby('planet_name')['spectral_type'].agg(first_non_zero)
avg_data_reset_index = avg_data.reset_index()
avg_data_with_spectral = pd.merge(avg_data_reset_index, spectral, on='planet_name', how='left')
avg_data_with_spectral

In [ ]:
avg_data_with_spectral.rename(columns={'spectral_type_y' : 'spectral_type'}, inplace=True)
avg_data = avg_data_with_spectral
#avg_data.drop(columns=['spectral_type_x'], inplace=True)
avg_data.head()

In [ ]:
# Checking NAN counts for each row.
for column in avg_data.columns:
    nan_count = avg_data[column].isna().sum()
    print(f"Number of NaN values in '{column}': {nan_count}")

In [ ]:
#Remove rows that have controversy
avg_data_filtered = avg_data.loc[avg_data['controv_flag'] != 1]
avg_data_filtered.reset_index(drop=True, inplace=True)
avg_data_filtered['controv_flag'].nunique()

In [ ]:
proj_data = avg_data_filtered
proj_data.head()

In [ ]:
proj_data2 = proj_data.drop(columns=['controv_flag'])
proj_data2.head()

# Factoring in Earth

Mass = 5.9722×10^24 kg

Eccentricity =0.0034 and up to 0.058. AVG = 0.0307

Radius = 3,958.8 miles

Orbital Period = 365

Semi-major Axis of Orbit = 149,598,023 km (or 92,955,902 miles)

Insolation Flux = 1400 W/m

Equilibrium Temp = 255 K

Spectral Type = G-type (G2-V)

Solar Mass = 1.989 × 10^30 kg

Sol Radius = 695,700 kilometres

In [ ]:
columns = ['planet_name', 'num_stars', 'orbital_period', 'planet_semi-major_axis', 'planet_radius', 'planet_mass', 'eccentricity', 'insolation_flux', 'equi_temp', 'stellar_temp', 'stellar_radius', 'stellar_mass', 'distance']
earth = pd.DataFrame(columns=columns)
earth_data = {'planet_name' : 'Earth',
              'num_stars' : 1,
              'orbital_period' : 365,
              'planet_semi-major_axis' : 1,
              'planet_radius' : 1,
              'planet_mass' : 1,
              'eccentricity' : 0.0307,
              'insolation_flux' : 0,
              'equi_temp' : 255,
              'stellar_temp' : 5800,
              'stellar_radius' : 1,
              'stellar_mass' : 1,
              'distance' : 0
}
earth = earth.append(earth_data, ignore_index=True)
earth

# Steps

1) Calculate planets in habitable zone.

    a) Planetary Equilibrium Temp is easiest. 175 K - 270 K
    
    b) Semi-major axis (distance from spectral body) and spectral type / stellar temp to estimate equi_temp
    


2) Clean data with planets that do not reside in the habitable zone

In [ ]:
# How many planets are habitable based upon our current count?
k_data = proj_data2
habitable = k_data[(k_data['equi_temp'] >= 175) & (k_data['equi_temp'] <= 270)]
count = habitable['planet_name'].count()
missing = k_data['equi_temp'].isna().sum()
print(count, "planets with equilibrium temp within acceptable habitable zone parameters.", missing, "NAN values")

In [ ]:
habitable

In [ ]:
# counts after removal of controversies
proj_data2.count()

# Solving Missing Equilibrium Temperature

In [ ]:
# Does insolation_flux help?
miss_temp = k_data[k_data['equi_temp'].isna()]
#miss_temp.count()
# 93 counts with insolation flux
insol_flux = k_data[~k_data['insolation_flux'].isna()]
insol_flux.head()

In [ ]:
insol_habit = insol_flux[(insol_flux['insolation_flux'] >= 175) & (insol_flux['insolation_flux'] <= 300)]
both_habit = insol_habit[(insol_habit['equi_temp'] < 800)]
both_habit

# Conclusion on Insolation Flux

Useless parameter, the planet's who's insolation flux have an equilibrium temperature that is too high to sustain life.

In [ ]:
#removing insolation flux from dataframe
k_data = k_data.drop(columns=['insolation_flux'])
no_flux = k_data
no_flux.head()

# Continuation of finding NAN equi_temp

Let $T_{s} = $ temperature of star, $R = $ radius of star, $a = $ semi-major axis, and $A_{B} = $ Bond Albedo

$T_{eq} = T_{s}\sqrt{\frac{R}{2a}}(1-A_{B})^{\frac{1}{4}}$

However, we don't have Bond Albedo, and if we did then the scientists that have been focused on this effort would have already done the math...

BUT WE CAN ESTIMATE!!!

Earth's Bond Albedo is 0.29

Bond Albedo is primarily affected by atmosphere. More clouds, more albedo.

Would it be wrong to take an average of all of our planet's bond albedos as our $A_{B}$?

In [ ]:
no_flux.head()

Using 11 Com b as a trial run. $0.3535 = $ average bond albedo sol system planets. Acknowledging this probably only works for G Type stars that are similar to Sol.

Let $T_{s} = 4808$  $R = 11395566$, $a = 183407176$, and $A_{B} = 0.3535$


$T_{eq} = (4808)\sqrt{\frac{11395566}{2 * 183407176}}(1 - 0.3535)^{\frac{1}{4}}$

$T_{eq} = (4808)\sqrt{\frac{11395566}{366814352}}(0.6465)^{\frac{1}{4}}$

$T_{eq} = 847.44(0.89669)$

$T_{eq} = 759.89$

Does this seem right? $1.22$ times further from star. The star is a giant but is colder than Sol. $16$ times larger radius really 

Let's try this same calculation with Earth and see if it's at least similar.

Let $T_{s} = 5800$  $R = 695700$, $a = 149598023$, and $A_{B} = 0.3535$


$T_{eq} = (5800)\sqrt{\frac{695700}{2 * 149598023}}(1 - 0.3535)^{\frac{1}{4}}$

$T_{eq} = 279.68(0.89669)$

$T_{eq} = 250.79$

Actual $= 255$

Error $= |Actual - Estimate| = 4.21$

While that is not how one is supposed to calculate inter-solar equilibrium temperature, that was closer than what I was expecting.

In [ ]:
#Function to fill in missing equi_temp
def calc_equi_temp(row):
    if np.isnan(row['equi_temp']): 
        stellar_temp = row['stellar_temp']
        stellar_radius = row['stellar_radius']
        a = row['planet_semi-major_axis']
        stellar_rad = stellar_radius * 695700
        b = a * 149598023
        #print(stellar_temp)
        #print(stellar_rad)
        #print(b)
        return stellar_temp * np.sqrt(stellar_rad / (2 * b)) * 0.89669
    else:
        return row['equi_temp']

In [ ]:
trial = no_flux.copy(deep=True)

In [ ]:
trial['equi_temp'] = trial.apply(calc_equi_temp, axis=1)
trial.head()

In [ ]:
#Remove the two White Dwarves, did the math, wouldn't matter to the data
trial = trial[~trial['spectral_type'].str.contains('WD')]

In [ ]:
# Now let's see how many stars we have in the habitable.
habitable2 = trial[(trial['equi_temp'] >= 175) & (trial['equi_temp'] <= 270)]
count2 = habitable2['planet_name'].count()
missing2 = trial['equi_temp'].isna().sum()
print(count2, "planets with equilibrium temp within acceptable habitable zone parameters.", missing2, "NAN values")

In [ ]:
still_missing = trial[trial['equi_temp'].isna()]
still_missing.count()

# Problem

If stellar temp, stellar radius, or planet_sma are NAN then unable to find equilibrium, meaning 600 possible planets are still unknown.

For stellar temp, since we know the spectral type in some instances we can further approximate. 

Will use this page for temperature. https://sites.uni.edu/morgans/astro/course/Notes/section2/spectraltemps.html

For Stellar Radius we may have to interpolate.

Rows without spectral types will be removed below.

In [ ]:
#Removes rows missing spectral type. Lose ~400 rows
still_missing = still_missing[still_missing['spectral_type'] != '0']
still_missing.count()

In [ ]:
#separating spectral_type into two
split_columns = still_missing['spectral_type'].str.split(expand=True)
still_missing['spectral_type'] = split_columns[0]
still_missing['spectral_subtype'] = split_columns[1] if len(split_columns.columns) > 1 else np.nan
still_missing.count()

In [ ]:
# function to fill in missing stellar_temp
def fill_stellar_temp(df):
    III_mapping = {
        #O type Full Info
        'O5': 54000,
        'O6': 45000,
        'O7': 43300,
        'O8': 40600,
        'O9': 37800,
        
        #B type Full Info
        'B0': 29200,
        'B1': 23000,
        'B2': 21000,
        'B3': 17600,
        'B5': 15200,
        'B6': 14300,
        'B7': 13500,
        'B8': 12300,
        'B9': 11400,
        
        #A type Full Info
        'A0': 9600,
        'A1': 9330,
        'A2': 9040,
        'A3': 8750,
        'A4': 8480,
        'A5': 8310,
        'A7': 7920,
        
        #F type Full Info
        'F0': 7350,
        'F2': 7050,
        'F3': 6850,
        'F5': 6700,
        'F6': 6550,
        'F7': 6400,
        'F8': 6300,
        
        #G type Full Info
        'G0': 6050,
        'G1': 5930,
        'G2': 5800,
        'G5': 5660,
        'G8': 5440,
        
        #K type Full Info
        'K0': 5240,
        'K1': 5110,
        'K2': 4960,
        'K3': 4800,
        'K4': 4600,
        'K5': 4400,
        'K7': 4000,
        
        #M type Full Info
        'M0': 3750,
        'M1': 3700,
        'M2': 3600,
        'M3': 3500,
        'M4': 3400,
        'M5': 3200,
        'M6': 3100,
        'M7': 3900,
        'M8': 2700
    }
    
    for index, row in df.iterrows():
        if pd.isna(row['stellar_temp']):
            spectral_type = row['spectral_type']
            if spectral_type in III_mapping:
                df.at[index, 'stellar_temp'] = III_mapping[spectral_type]
    
    return df

filled_trial = fill_stellar_temp(still_missing)
filled_trial.head(50)

In [ ]:
#nan_count = filled_trial['stellar_temp'].isna().sum()
#nan_count

In [ ]:
#filled_trial[filled_trial['stellar_temp'] == 4960].count()

In [ ]:
#trial[trial['stellar_temp'] == 4960].count()

In [ ]:
filled_trial = filled_trial.dropna(subset=['stellar_temp'])

In [ ]:
filled_trial.count()

In [ ]:
# Okay we now have 187 additional stellar temps calculated. 
# Let's fill in missing Semi-Major_axis based upon orbital period
filled_trial.loc[filled_trial['planet_semi-major_axis'].isna(), 'planet_semi-major_axis'] = (filled_trial['orbital_period']**2)**(1/3)
filled_trial.count()
#filled_trial.head(50)

In [ ]:
# Now we see we filled in missing planet_sma based upon Kepler's Third Law.
# Once more we can run our calc_equi_temp on 42 exo-planets since we have 42 stellar radius
# However that leaves us with 145 planets still unable to calculate due to missing stellar radius.
filled_trial['equi_temp'] = filled_trial.apply(calc_equi_temp, axis=1)
filled_trial.count()

In [ ]:
habitable3 = filled_trial[(filled_trial['equi_temp'] >= 175) & (filled_trial['equi_temp'] <= 270)]
count3 = habitable3['planet_name'].count()
missing3 = filled_trial['equi_temp'].isna().sum()
print(count3, "planets with equilibrium temp within acceptable habitable zone parameters.", missing3, "NAN values")

In [ ]:
# Merge three habitable dataframes

habitable_true = pd.concat([habitable, habitable2, habitable3], ignore_index=True)
habitable_true.count()

In [ ]:
# Secondary clean up due to messy data being merged.

# Remove Flux once again
habitable_true.drop(columns=['insolation_flux'])

# Split spectral type
split_columns = habitable_true['spectral_type'].str.split(expand=True)
habitable_true['spectral_type'] = split_columns[0]
habitable_true['spectral_subtype'] = split_columns[1] if len(split_columns.columns) > 1 else np.nan

In [ ]:
# Something is wrong. Habit1 = 78, Habit2 = 212, Habit3 = 3
# Found it. Accidentally ran concat more than once. 
# But also found out each habitable had duplicates that needed to be removed.
habitable_true.drop_duplicates(subset='planet_name', inplace=True)

habitable_true.reset_index(inplace=True)
habitable_true

In [ ]:
habitable_true["planet_density"] = habitable_true["planet_mass"] / (habitable_true["planet_radius"] ** 3)
habitable_true["planet_surface_gravity"] = G * (habitable_true["planet_mass"] * M_earth )/ ((habitable_true["planet_radius"] * R_earth) ** 2)
habitable_true["planet_dist_to_sellar_radius_ratio"] = habitable_true["planet_semi-major_axis"] / habitable_true["stellar_radius"]

habitable_true.plot(x="planet_semi-major_axis", y="planet_radius", kind="scatter", logx=True, logy=True)
habitable_true.plot(x="planet_semi-major_axis", y="planet_mass", kind="scatter", logx=True, logy=True)
habitable_true.plot(x="stellar_radius", y="planet_density", kind="scatter", logx=True, logy=True)
habitable_true.plot(x="insolation_flux", y="planet_mass", kind="scatter", logx=True, logy=True)
habitable_true.plot(x="planet_semi-major_axis", y="planet_surface_gravity", kind="scatter", logx=True, logy=True)
habitable_true.plot(x="planet_dist_to_sellar_radius_ratio", y="planet_surface_gravity", kind="scatter", logx=True, logy=True)
habitable_true.corr()

habitable_true.to_csv("habitable_true.csv")


# eq_temp 
# density
# mass

# error analysis - how close to earth